In [ ]:
# default_exp core
%load_ext lab_black

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import os
import sys
import shlex
import argparse
import pandas as pd
from pathlib import Path
from art import text2art
from collections import namedtuple
from subprocess import Popen, PIPE


if os.name == "nt":
    git_exe = r"C:\Program Files\Git\cmd\git.exe"
else:
    git_exe = "/usr/bin/git"
os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = git_exe

lr = "\n"
python = sys.executable
home = Path(os.path.expanduser("~"))
blogslicer = Path(home / Path("github/blogslicer/blogslicer/core.py"))
if hasattr(__builtins__, "__IPYTHON__") or __name__ != "__main__":
    is_jupyter = True
    from IPython.display import display, Markdown, HTML

    file = "sites.csv"
    apex = "/mnt/c/Users/mikle/github/MikeAtEleven.com"
    # apex = ""
else:
    is_jupyter = False
    aparser = argparse.ArgumentParser()
    add_arg = aparser.add_argument
    add_arg("-f", "--file", required=True)
    add_arg("-x", "--apex", required=False)
    args = aparser.parse_args()
    file = args.file
    apex = args.apex


def fig(text, font="Standard"):
    if is_jupyter:
        text_br = text2art(text, font=font).replace(lr, "<br/>")
        text_html = f'<pre style="white-space: pre;">{text_br}</pre>'
        display(HTML(text_html))
    else:
        print(text2art(text, font))


def flush(std):
    for line in std:
        line = line.strip()
        if line:
            print(line)
            sys.stdout.flush()


def git(cwd, args):
    cmd = [git_exe] + shlex.split(args)
    print(f"COMMAND: << {shlex.join(cmd)} >>")
    process = Popen(
        args=cmd,
        cwd=cwd,
        stdout=PIPE,
        stderr=PIPE,
        shell=False,
        bufsize=1,
        universal_newlines=True,
    )
    flush(process.stdout)
    flush(process.stderr)

fig("Making Sites...")

file_obj = Path(file)
df = pd.read_csv(file_obj, delimiter="|")
df = df.applymap(lambda x: x.strip())
df.columns = [x.strip() for x in df.columns]
if apex:
    apex = Path(apex).name
    df = df[df["apex"] == apex]
Site = namedtuple("Site", "path, apex, title, gaid, tagline")
print(f"INTERPRETER: << {python} >>")
print(f"SLICER: << {blogslicer} >>")
for index, series in df.iterrows():
    site = Site(**series.to_dict())
    fig(site.apex, font="Cybermedium")
    here = Path(home / site.path)
    [x.unlink() for x in Path(here / "_posts/").glob("*")]
    cmd = f'{python} {blogslicer} -p {here} -t "{site.title}" -s "blog" -a "Mike Levin"'
    print(cmd, end="\n\n")
    with Popen(args=cmd, cwd=here, stdout=PIPE, stderr=PIPE, shell=True) as pout:
        for line in pout.stdout.readlines():
            print(line.decode().strip())
    fig('Github...')
    git(here, "add _posts/*")
    git(here, "add assets/images/*")
    git(here, 'commit -am "Pushing to Github..."')
    git(here, "push")
fig("Done!")

INTERPRETER: << C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe >>
SLICER: << C:\Users\mikle\github\blogslicer\blogslicer\core.py >>


C:\Users\mikle\AppData\Roaming\jupyterlab-desktop\jlab_server\python.exe C:\Users\mikle\github\blogslicer\blogslicer\core.py -p C:\Users\mikle\github\MikeAtEleven.com -t "Mike At Eleven" -s "blog" -a "Mike Levin"

C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-06-06-post-23.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-06-05-post-22.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-07-post-21.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-06-post-20.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-05-post-19.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-18.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-17.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-16.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-15.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-14.md
C:\Users\mikle\github\MikeAtEleven.com/_posts/2022-05-04-post-13.md
C:\Users\mikle\github\MikeAtEleven.com

COMMAND: << 'C:\Program Files\Git\cmd\git.exe' add '_posts/*' >>
COMMAND: << 'C:\Program Files\Git\cmd\git.exe' add 'assets/images/*' >>
COMMAND: << 'C:\Program Files\Git\cmd\git.exe' commit -am 'Pushing to Github...' >>
On branch main
Your branch is up to date with 'origin/main'.
nothing to commit, working tree clean
COMMAND: << 'C:\Program Files\Git\cmd\git.exe' push >>
Everything up-to-date


In [ ]:
!nbdev_build_lib

Converted 00_core.ipynb.
Converted index.ipynb.


In [ ]:
!nbdev_clean_nbs